In [1]:
from IPython.display import display
from os.path import isfile, join, exists
from os import listdir, makedirs, walk, remove, getlogin
from nilearn import image, plotting
import pandas as pd
import re
import numpy as np
import fileinput

/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [87]:
#---- Set subject specific variables ----#

# Set subject ID
subid = 'impr_g2_s01_030818'

# manually set scan num, type, conds
anat_num = [3]
fieldmap_nums = [8,9]
pos_epi_num = [4]
neg_epi_num = [6]
mix_epi_num = [5]
pie_epi_num = [7]
conds = ['pos2', 'neg2', 'mixed2', 'pieman']



,preprocdir,scanNum,scanType
pos2,run1,4,epi
neg2,run2,6,epi
mixed2,run3,5,epi
pieman,run4,7,epi
se_fieldmap_ap,0,8,fieldmap
se_fieldmap_pa,0,9,fieldmap
anat,0,3,anat


In [88]:
#---- Set experiment-specific parameters ----#

# Set projdir
projdir = '/mnt/bucket/labs/hasson/mai/projects/impressions'

# Set data dir
datadir = join(projdir, 'data')

# Set code dir
codedir = join(projdir, 'code', 'preproc', 'bash')

# Set template dir
templatedir = join(projdir, 'code', 'preproc', 'template')

# Set niftidir
niftidir = join(datadir, subid, 'raw', 'nii')

# Set outputdir
anat_outdir = join(datadir, subid, 'data', 'niftis')
if not exists(anat_outdir):
    print(anat_outdir + ' does not exist, creating')
    makedirs(anat_outdir)
    

In [ ]:
#---- Copy and modify preproc scripts for subject ----#

# Setup script: 01_make_preprocFiles.py
template_script = join(templatedir, '01_make_preprocFiles_template.py')
subj_script = join(preproc_script_dir, '01_make_preprocFiles.py')
!cp {template_script} {subj_script}
#output = subprocess.check_output(['cp', template_script, subj_script])

for line in fileinput.input(subj_script, inplace=1):
    line = re.sub('###SUB_ID###', "'" + subid + "'", line.rstrip())
    line = re.sub('###PROJ_DIR###', "'" + projdir + "'", line.rstrip())
    line = re.sub('###REF_SCAN###', "'" + ref_scan + "'", line.rstrip())
    print(line)
fileinput.close()

# Setup script: 02_run_preproc.py
template_script = join(templatedir, '02_run_preproc_template.py')
subj_script = join(preproc_script_dir, '02_run_preproc.py')
!cp {template_script} {subj_script}
#output = subprocess.check_output(['cp', template_script, subj_script])


for line in fileinput.input(subj_script, inplace=1):
    line = re.sub('###SUB_ID###', "'" + subid + "'", line.rstrip())
    line = re.sub('###CONDS###', str(conds_order_std), line.rstrip())
    line = re.sub('###PROJ_DIR###', "'" + projdir + "'", line.rstrip())
    print(line)
fileinput.close()


In [97]:
#---- Save scan info -----#

# organize scan info
anat = pd.DataFrame({'preprocDir': [0], 'scanNum': anat_num, 'scanType': 'anat'}, index=['anat'])

fieldmaps = pd.DataFrame({'preprocDir': [0,0], 'scanNum': fieldmap_nums, 'scanType': ['fieldmap', 'fieldmap']}, 
                        index=['se_fieldmap_ap', 'se_fieldmap_pa'])

pos_epi = pd.DataFrame({'preprocDir': 'run1', 'scanNum': pos_epi_num, 'scanType': 'epi'}, index=[conds[0]])
neg_epi = pd.DataFrame({'preprocDir': 'run2', 'scanNum': neg_epi_num, 'scanType': 'epi'}, index=[conds[1]])
mix_epi = pd.DataFrame({'preprocDir': 'run3', 'scanNum': mix_epi_num, 'scanType': 'epi'}, index=[conds[2]])
pie_epi = pd.DataFrame({'preprocDir': 'run4', 'scanNum': pie_epi_num, 'scanType': 'epi'}, index=[conds[3]])

df = pd.concat([pos_epi, neg_epi, mix_epi, pie_epi, fieldmaps, anat])

# automatically get scan names
niftiNames = np.array([])
niftis_all = listdir(niftidir)

for ind,row in df.iterrows():
    scanNum = row['scanNum']
    for filename in niftis_all:
        if str(scanNum) + '_' in filename:
            niftiNames = np.append(niftiNames, filename)
            break

# add nifti names to dataframe
df['niftiName'] = niftiNames

# add anatomies to scan info
newNiftis = pd.DataFrame({'niftiName': [subid + '_t1w_reorient.nii.gz', subid + '_t1w_reorient_brain.nii.gz'],
                          'scanNum': [0,0], 'scanType': ['anat', 'anat'],
                          'preprocDir': [0,0]}, index=['anat_reorient', 'anat_brain'] )

# Save scan info
df = df.append(newNiftis)
df.to_csv(join(datadir, subid, 'scanInfo.csv'))

print(df)
print('\n**** done!\n')

reorient brain to std ...
done!
                                                   niftiName preprocDir  \
pos2                 04_EPI-2mm-CMRR-PPF78-1500TR-MB4.nii.gz       run1   
neg2                 06_EPI-2mm-CMRR-PPF78-1500TR-MB4.nii.gz       run2   
mixed2               05_EPI-2mm-CMRR-PPF78-1500TR-MB4.nii.gz       run3   
pieman               07_EPI-2mm-CMRR-PPF78-1500TR-MB4.nii.gz       run4   
se_fieldmap_ap                 08_SpinEchoFieldMap_AP.nii.gz          0   
se_fieldmap_pa                 09_SpinEchoFieldMap_PA.nii.gz          0   
anat                              03_T1w_MPR_1iso_iP2.nii.gz          0   
anat_reorient         impr_g2_s01_030818_t1w_reorient.nii.gz          0   
anat_brain      impr_g2_s01_030818_t1w_reorient_brain.nii.gz          0   

                scanNum  scanType  
pos2                  4       epi  
neg2                  6       epi  
mixed2                5       epi  
pieman                7       epi  
se_fieldmap_ap        8  fieldmap  
s

In [ ]:
# # ---- Reorient and extract brain ----------------------------------------------
anat_orig = join(niftidir, df.loc['anat']['niftiName'])
anat_reorient = join(anat_outdir, subid + '_t1w_reorient.nii.gz')
#output = subprocess.check_output([join(codedir, 'run_reorient.sh'), anat_orig, anat_reorient])
!{join(codedir, 'run_reorient.sh')} {anat_orig} {anat_reorient}